In [1]:
import sys
sys.path.append('..')

from scripts.GenrationGI0 import partitioned_gi0_image, generate_multiple_images, mixed_dataset
from scripts.autoencoders import InMemoryImageDataset, ConfigurableAutoencoder

import pandas as pd
import torch
from torchvision import transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
from torchsummary import summary
import numpy as np
import matplotlib.pyplot as plt
import cv2
from omegaconf import OmegaConf
OmegaConf.register_new_resolver("eval", eval)

Elegir el archivo de configuración correspondiente:

In [2]:
config_name = 'config_1' # Elegir

config_path = f'configs/{config_name}.yaml'
config = OmegaConf.load(config_path)
config

{'training': {'n': 50000, 'n_cuad_lado': [2], 'pixeles_cuad': [25], 'num_epochs': 20, 'batch_size': 64, 'learning_rate': 0.001, 'scheduler_name': 'None', 'scheduler_params': 'None'}, 'model': {'encoding_dim': 32, 'loss_function': 'mse', 'optimizer': 'adam'}, 'encoder': {'layers': [{'type': 'conv2d_transpose', 'filters': 1, 'kernel_size': 3, 'stride': 1, 'padding': 1, 'activation': 'sigmoid'}, {'type': 'maxpool2d', 'pool_size': 4}, {'type': 'flatten'}, {'type': 'dense', 'dim': 128, 'activation': 'relu'}, {'type': 'dense', 'dim': '${model.encoding_dim}'}]}, 'decoder': {'layers': [{'type': 'dense', 'dim': 128, 'activation': 'relu'}, {'type': 'dense', 'dim': "${eval:'(${training.n_cuad_lado[0]} * ${training.pixeles_cuad[0]})**2'}", 'activation': 'sigmoid'}, {'type': 'unflatten', 'dim1': "${eval:'${training.n_cuad_lado[0]} * ${training.pixeles_cuad[0]}'}", 'dim2': "${eval:'${training.n_cuad_lado[0]} * ${training.pixeles_cuad[0]}'}"}]}, 'testing': {'n': 1000, 'batch_size': 32}}

Cargo el autoencoder ya entrenado:

In [3]:
# 1. Crear una instancia del modelo (debe tener la misma arquitectura)
autoencoder_cargado = ConfigurableAutoencoder(config=config)
# 2. Carga los parámetros
autoencoder_cargado.load_state_dict(torch.load(f'data/trained_models/{config_name}.pth'))
# 3. Modo evaluación (cuando lo use para inferencia)
autoencoder_cargado.eval()

C:\Users\Bianca\AppData\Local\Temp\ipykernel_18812\2579078806.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  autoencoder_cargado.load_state_dict(torch.load(f'data/train

ConfigurableAutoencoder(
  (encoder): Sequential(
    (0): ConvTranspose2d(1, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): Sigmoid()
    (2): MaxPool2d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
    (3): Flatten(start_dim=1, end_dim=-1)
    (4): Linear(in_features=144, out_features=128, bias=True)
    (5): ReLU()
    (6): Linear(in_features=128, out_features=32, bias=True)
  )
  (decoder): Sequential(
    (0): Linear(in_features=32, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=2500, bias=True)
    (3): Sigmoid()
    (4): Unflatten(dim=1, unflattened_size=(1, 50, 50))
  )
)

Genero dataset de testeo:

In [9]:
n = config['testing']['n']
batch_size = config['testing']['batch_size']
n_cuad_lado = config['training']['n_cuad_lado']
pixeles_cuad = config['training']['pixeles_cuad']

test_g, test_gi, test_gI0 = mixed_dataset(
    n_total = n,
    generate_multiple_images = generate_multiple_images,
    conjunto_n_cuad_lado = n_cuad_lado,
    conjunto_pixeles_cuad = pixeles_cuad,
    ratios = config.training.get('ratio',[1]),
)